In [2]:
import glob
from os.path import basename, splitext
from inaGVAD.trs_utils import trs2df, parsedf
from pyannote.core import Annotation, Segment, Timeline


import pandas as pd

In [7]:
def map_gender(s):
    # MAP speech labels to gender label
    #
    
    if s == '':
        return 'nonspeech'
    
    labels = list(set([e[0] for e in s.split('+') if e[0] in 'HFI']))
    
    if len(labels) == 0:
        return 'nonspeech'
    
    if len(labels) > 1:
        print(s)
        return 'undefspeech'
    
    if s[0] == 'I':
        return 'undefspeech'
    
    if s[0] == 'H':
        return 'male'
    
    if s[0] == 'F':
        return 'female'

def map_vad(s):
    if s == '':
        return 'nonspeech'
    if s[0] in 'HFI':
        return 'speech'
    else:
        return 'nonspeech'

def df2annotation(df):
    df['annot_gender'] = df.label.map(map_gender)
    df['annot_vad'] = df.label.map(map_vad)

    an = Annotation()
    uem = Timeline()
    uem.add(Segment(df.start[0], df.stop[len(df)-1]))
    vad = Annotation()

    for t in df.itertuples():
        seg = Segment(t.start, t.stop)
        vad[seg] = t.annot_vad
        if t.annot_gender in ['male', 'female']:
            an[seg] = t.annot_gender
        if t.annot_gender == 'undefspeech':
            uem = uem.extrude(seg)

    return vad.support(), an.support(), uem.support()


In [16]:
def annot2df(an):
    lrec = []
    for seg, track, label in an.itertracks(yield_label=True):
        lrec.append((seg.start, seg.end, label))
    return pd.DataFrame.from_records(lrec, columns=['start', 'stop', 'label'])

In [3]:
for f in glob.glob('../../annotations/trs/*.trs'):
    base = splitext(basename(f))[0]
    break
    vad, gender, uem = df2annotation(trs2df(f))
    with open('../../annotations/gender_uem/%s.uem' % base, 'wt') as fid:
        uem.write_uem(fid)
    annot2df(gender).to_csv('../../annotations/gender/%s.csv' % base, index=False)
    annot2df(vad).to_csv('../../annotations/vad/%s.csv' % base, index=False)
    print(base)

In [4]:
df = trs2df(f)

In [5]:
df

,start,stop,label,dur
0,0.000,0.300,MU2,0.300
1,0.300,1.776,HA,1.476
2,1.776,3.558,MU2,1.782
3,3.558,4.012,HA,0.454
4,4.012,6.073,MU2,2.061
5,6.073,6.422,HA,0.349
6,6.422,7.888,,1.466
7,7.888,8.563,HA,0.675
8,8.563,9.987,MU2,1.424
9,9.987,11.116,HA,1.129


In [8]:
df['vad'] = df.label.map(map_vad)

In [9]:
df

,start,stop,label,dur,vad
0,0.000,0.300,MU2,0.300,nonspeech
1,0.300,1.776,HA,1.476,speech
2,1.776,3.558,MU2,1.782,nonspeech
3,3.558,4.012,HA,0.454,speech
4,4.012,6.073,MU2,2.061,nonspeech
5,6.073,6.422,HA,0.349,speech
6,6.422,7.888,,1.466,nonspeech
7,7.888,8.563,HA,0.675,speech
8,8.563,9.987,MU2,1.424,nonspeech
9,9.987,11.116,HA,1.129,speech
